In [102]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cv2 
import PIL.Image as Image
import torch
import torchvision
# from torchvision.io import read_image
from torch.utils.data import Dataset
import torchvision.transforms as T
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets

from time import time

In [103]:
device = "cpu" 
print(f"Using {device} device")

Using cpu device


In [117]:
IMAGE_SIZE = 224
transforms = T.Compose(
            [T.Resize((IMAGE_SIZE,IMAGE_SIZE)),
            # T.ToTensor(),
            # T.Normalize([0.5,0.5,0.5],[0.25,0.25,0.25]),            
            ])

In [105]:
#Using albumentations
# import albumentations as A
# from albumentations.pytorch import ToTensorV2

# transforms_album =   A.Compose([
#     A.Resize(128,128),
#     A.HorizontalFlip(p=0.38),
#     A.Normalize([0.5,0.5,0.5],[0.25,0.25,0.25]),
#     ToTensorV2()
#     ])


In [118]:
class DogCatDataset(Dataset):
  def __init__ (self, annotations_file, img_dir, transform=transforms,target_transform=None):
    self.img_labels = pd.read_csv(annotations_file, sep=(','))
    self.img_dir = img_dir
    self.transform = transform
    self.target_transform = target_transform
  def __len__(self):
    return len(self.img_labels)
  def __getitem__(self, index):
    img_path = os.path.join(self.img_dir, self.img_labels.iloc[index,0])
    image = cv2.imread(img_path)
    label = self.img_labels.iloc[index,1]
    
    if self.transform == transforms:
            image = self.transform(image)
    # elif self.transform == transforms_album:
    #     augmented = self.transform(image=image)
    #     image = augmented['image']
    return image,label

In [119]:
batch_size = 4

In [120]:
images='/home/sandbox-2/Documents/Gopal_office_file/dogs-vs-cats/dataset_3000img'
label="/home/sandbox-2/Documents/Gopal_office_file/dogs-vs-cats/dataset3000.csv"

dataset = DogCatDataset(annotations_file=label,img_dir=images)

train_per = int(len(dataset)*0.65)
val_per = int(len(dataset)*0.25)
test_per = len(dataset)-train_per-val_per

train_data, validation_data, test_data= torch.utils.data.random_split(dataset,[train_per,val_per,test_per])    #16250,3750,5000

train_loader = torch.utils.data.DataLoader(train_data,batch_size=batch_size,shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_data,batch_size=batch_size,shuffle=False)
test_loader = torch.utils.data.DataLoader(test_data,batch_size=batch_size,shuffle=False)

print(train_per, val_per, test_per)

1950 750 300


In [121]:
print(f"Lenth of Train data ----->{len(train_data)} \
        \nLenth of Validation data ----->{len(validation_data)}\
       # \nLenth of Test data ----->{len(test_data)}")

Lenth of Train data ----->1950         
Lenth of Validation data ----->750       # 
Lenth of Test data ----->300


In [ ]:
img = train_data[0][1]
# img = train_data[0]
# print(img)
# img = T.ToPILImage()(img).convert("RGB")
# img.show()

In [39]:
model = nn.Sequential(
    nn.Conv2d(3,48,3,stride=2,padding=1),nn.ReLU(),
    nn.MaxPool2d(3,3),

    nn.Conv2d(48,96,kernel_size=3,padding=2), nn.ReLU(),
    nn.MaxPool2d(3,2),

 # nn.Conv2d(256,384,3,1), nn.ReLU(),
    nn.Conv2d(96,96,3,1), nn.ReLU(),
    nn.MaxPool2d(3,2),
    nn.Conv2d(96,192,3,1), nn.ReLU(),
    nn.MaxPool2d(3,2),
    
    nn.Flatten(),

    nn.Linear(768,384),nn.ReLU(),
    nn.Dropout(p=0.7),
    nn.Linear(384,192),nn.ReLU(),
    nn.Dropout(p=0.7),
    nn.Linear(192,2)  
)
model = model.to(device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0000001)
epoch = 500

In [ ]:
train_loss = []
valiadtion_loss = []
train_accuracy = []
test_accuracy = []
epoch = epoch

best_loss = np.inf
early_stopping_iter = 10
early_stopping_counter = 10

for t in range(epoch):
    start = time()
    print(f"Epoch {t+1}\n .........................")
    
    for data_loader in [train_loader,validation_loader]:
        if data_loader == train_loader:
            model.train()
        else:
            model.eval()
        losses=[]
        size = len(data_loader.dataset)
        for batch, (X,y) in enumerate(data_loader):                 
            #computr prediction and loss
            pred = model(X)
            loss = loss_fn(pred,y)
            losses.append(loss.item())
            if data_loader == train_loader:
            #Backprobagation
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        if data_loader == train_loader:
            train_loss.append(np.mean(losses))
        if data_loader == validation_loader:
            valiadtion_loss.append(np.mean(losses))
            if valiadtion_loss < best_loss:
                best_loss = valiadtion_loss
            else:
                early_stopping_counter += 1

    for data_loader in [train_loader,test_loader]:
        model.eval()          
        correct=0.0
        total=0.0
        size = len(data_loader.dataset)
        for batch, (X,y) in enumerate(data_loader):
            X, y = X, y           
            #computr prediction and loss
            pred = model(X)
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
      
        if data_loader == test_loader:
            test_acc=correct/size*100
            test_accuracy.append(test_acc)
        if data_loader == train_loader:
            train_acc=correct/size*100
            train_accuracy.append(train_acc)
    print((time()-start)/60,"mins")

    if early_stopping_counter > early_stopping_iter:
        break



In [ ]:
plt.plot(np.arange(epoch),train_loss,label='Train Loss')
# plt.plot(np.arange(epoch),valiadtion_loss,label='Valiadtion Loss')
plt.xlabel('Epoch')
plt.ylabel('Losses')
plt.title("Losses Plot")
plt.legend()

In [ ]:
plt.plot(np.arange(epoch),train_accuracy,label='Train Accuracy')
plt.plot(np.arange(epoch),test_accuracy,label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title("Accuracy Plot")
plt.legend()

test with the saved weights

https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#sphx-glr-beginner-blitz-cifar10-tutorial-py

https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

In [ ]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    npimg = np.transpose(npimg, (1, 2, 0))
    npimg = npimg[...,::-1]
    plt.imshow(npimg)
    plt.show()
classes = ('cat','dog')

In [ ]:
path = '/home/sandbox-2/Documents/Gopal_office_file/dogs-vs-cats/model.pth'
dataiter = iter(train_loader)
image,label = dataiter.next()

print(image.shape)

#print images
# imshow(torchvision.utils.make_grid(image))
# print(' '.join(f'{classes[label[j]]:5s}' for j in range(batch_size)))

In [ ]:
model.load_state_dict(torch.load(path,map_location=torch.device('cpu')))

In [ ]:
outputs = model(image)
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}'
                              for j in range(4)))

In [34]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 89 %


In [36]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy for class: cat   is 89.2 %
Accuracy for class: dog   is 85.3 %


In [37]:
#Generic function to display predictions for a few images
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(test_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title(f'predicted: {classes[preds[j]]}, original{classes[labels[j]]}')
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
visualize_model(model)